In [9]:
from pyspark.sql import SparkSession 

sc = SparkSession.builder.appName("DataFrame").getOrCreate()

In [10]:
logs_interesse = [
    "Eleitor foi habilitado",
    "Voto confirmado para [Deputado Federal]",
    "Voto confirmado para [Deputado Estadual]",
    "Voto confirmado para [Senador]",
    "Voto confirmado para [Governador]",
    "Voto confirmado para [Presidente]",
    "Tecla indevida pressionada",
]
regex_interesse = [
    "Município:",
    "Zona Eleitoral:",
    "Local de Votação:",
    "Seção Eleitoral:",
]
regex_interesse = "|".join(regex_interesse)

In [11]:
import pyspark.sql.functions as F

DATA_PATH = "/app/data/"
input_file = DATA_PATH+"decodificado/*" 

source = sc.read.csv(input_file, sep="\t", header=False)
source = source.withColumn("arquivo", F.input_file_name())

24/03/16 17:53:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/16 17:53:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/16 17:53:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/16 17:53:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/16 17:53:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/16 17:53:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/16 17:53:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/16 17:53:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/16 17:53:40 WARN RowBasedKeyValueBatch: Calling spill() on

Selecionando amostra de 10% das urnas

In [12]:
distinto = source.select('arquivo').distinct()
amostra = distinto.sample(False, (2/distinto.count()),seed=1100)
source =  source.filter(F.col("arquivo").isin(*amostra.toPandas()['arquivo'].values))
print(f'Número de urnas selecionadas {amostra.count()}')

Número de urnas selecionadas 2


In [13]:
df = source.withColumnRenamed("_c0", "data").withColumnRenamed("_c4", "descricao")
df = df.select("data", "descricao")
df = df.withColumn("arquivo", F.input_file_name())
df = df.filter(
    F.col("descricao").isin(logs_interesse) | F.col("descricao").rlike(regex_interesse)
)

df = df.withColumn("data", F.to_timestamp(F.col("data"), "dd/MM/yyyy HH:mm:ss"))
df.show(truncate=False)

+-------------------+----------------------+------------------------------------------------------+
|data               |descricao             |arquivo                                               |
+-------------------+----------------------+------------------------------------------------------+
|2022-09-21 13:08:00|Município: 17558      |file:///app/data/decodificado/o00406-1755800440052.txt|
|2022-09-21 13:08:00|Zona Eleitoral: 0044  |file:///app/data/decodificado/o00406-1755800440052.txt|
|2022-09-21 13:08:00|Local de Votação: 1147|file:///app/data/decodificado/o00406-1755800440052.txt|
|2022-09-21 13:08:00|Seção Eleitoral: 0052 |file:///app/data/decodificado/o00406-1755800440052.txt|
|2022-09-21 13:13:21|Município: 17558      |file:///app/data/decodificado/o00406-1755800440052.txt|
|2022-09-21 13:13:21|Zona Eleitoral: 0044  |file:///app/data/decodificado/o00406-1755800440052.txt|
|2022-09-21 13:13:21|Seção Eleitoral: 0052 |file:///app/data/decodificado/o00406-1755800440052.txt|


In [14]:
from pyspark.sql.window import Window
from unidecode import unidecode

window = (
    Window.orderBy("data")
    .partitionBy("arquivo") #Paraleliza pelo arquivo
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

df_window = df
for nome_extracao in ["Município", "Zona Eleitoral", "Seção Eleitoral"]:
    nome_col = unidecode(nome_extracao).replace(" ", "_")
    df_window = df_window.withColumn(
        nome_col,
        F.when(
            F.col("descricao").contains(nome_extracao),
            F.regexp_extract(F.col("descricao"), rf"{nome_extracao}: (\d+)", 1).cast(
                "integer"
            ),
        ).otherwise(None),
    )

    df_window = df_window.withColumn(
        nome_col, F.last(nome_col, ignorenulls=True).over(window)
    )
df_window.show(truncate=False)

+-------------------+----------------------+------------------------------------------------------+---------+--------------+---------------+
|data               |descricao             |arquivo                                               |Municipio|Zona_Eleitoral|Secao_Eleitoral|
+-------------------+----------------------+------------------------------------------------------+---------+--------------+---------------+
|2022-09-19 13:39:40|Município: 17256      |file:///app/data/decodificado/o00406-1725600440147.txt|17256    |NULL          |NULL           |
|2022-09-19 13:39:40|Zona Eleitoral: 0044  |file:///app/data/decodificado/o00406-1725600440147.txt|17256    |44            |NULL           |
|2022-09-19 13:39:40|Local de Votação: 1058|file:///app/data/decodificado/o00406-1725600440147.txt|17256    |44            |NULL           |
|2022-09-19 13:39:40|Seção Eleitoral: 0147 |file:///app/data/decodificado/o00406-1725600440147.txt|17256    |44            |147            |
|2022-09-19 1

In [21]:
df_teclas = df_window.filter(~F.col("descricao").rlike(regex_interesse))
df_teclas = df_teclas.withColumn(
    "erros_tecla",
    F.when(F.col("descricao") == "Tecla indevida pressionada", 1).otherwise(0),
)

df_teclas = df_teclas.withColumn(
    "id_eleitor", F.when(F.col("descricao") == "Eleitor foi habilitado", 1).otherwise(0)
)
my_window = (
    Window.orderBy("data")
    .partitionBy("arquivo")
    .rowsBetween(Window.unboundedPreceding, 0)
)
df_teclas = df_teclas.withColumn("id_eleitor", F.sum("id_eleitor").over(my_window))


df_teclas.orderBy(["arquivo","data"], ascending=[1,1]).show(truncate=False)

+-------------------+----------------------------------------+------------------------------------------------------+---------+--------------+---------------+-----------+----------+
|data               |descricao                               |arquivo                                               |Municipio|Zona_Eleitoral|Secao_Eleitoral|erros_tecla|id_eleitor|
+-------------------+----------------------------------------+------------------------------------------------------+---------+--------------+---------------+-----------+----------+
|2022-10-02 08:01:46|Eleitor foi habilitado                  |file:///app/data/decodificado/o00406-1725600440147.txt|17256    |44            |147            |0          |1         |
|2022-10-02 08:02:26|Voto confirmado para [Deputado Federal] |file:///app/data/decodificado/o00406-1725600440147.txt|17256    |44            |147            |0          |1         |
|2022-10-02 08:02:44|Voto confirmado para [Deputado Estadual]|file:///app/data/decodificad

In [22]:
agrupado_eleitor = df_teclas.groupBy("arquivo", "id_eleitor").agg({"erros_tecla": "sum"}).orderBy(["arquivo","id_eleitor"], ascending=[1,1])
agrupado_eleitor.show(truncate=False)

+------------------------------------------------------+----------+----------------+
|arquivo                                               |id_eleitor|sum(erros_tecla)|
+------------------------------------------------------+----------+----------------+
|file:///app/data/decodificado/o00406-1725600440147.txt|1         |0               |
|file:///app/data/decodificado/o00406-1725600440147.txt|2         |4               |
|file:///app/data/decodificado/o00406-1725600440147.txt|3         |0               |
|file:///app/data/decodificado/o00406-1725600440147.txt|4         |0               |
|file:///app/data/decodificado/o00406-1725600440147.txt|5         |0               |
|file:///app/data/decodificado/o00406-1725600440147.txt|6         |0               |
|file:///app/data/decodificado/o00406-1725600440147.txt|7         |0               |
|file:///app/data/decodificado/o00406-1725600440147.txt|8         |0               |
|file:///app/data/decodificado/o00406-1725600440147.txt|9        

In [31]:
df_soma_tecla = df_teclas.join(agrupado_eleitor, on=['arquivo', 'id_eleitor'], how='left')
df_soma_tecla = df_soma_tecla.drop("erros_tecla")
df_soma_tecla.show(5)

+--------------------+----------+-------------------+--------------------+---------+--------------+---------------+----------------+
|             arquivo|id_eleitor|               data|           descricao|Municipio|Zona_Eleitoral|Secao_Eleitoral|sum(erros_tecla)|
+--------------------+----------+-------------------+--------------------+---------+--------------+---------------+----------------+
|file:///app/data/...|         1|2022-10-02 08:01:46|Eleitor foi habil...|    17256|            44|            147|               0|
|file:///app/data/...|         1|2022-10-02 08:02:26|Voto confirmado p...|    17256|            44|            147|               0|
|file:///app/data/...|         1|2022-10-02 08:02:44|Voto confirmado p...|    17256|            44|            147|               0|
|file:///app/data/...|         1|2022-10-02 08:02:57|Voto confirmado p...|    17256|            44|            147|               0|
|file:///app/data/...|         1|2022-10-02 08:03:07|Voto confirmado 

In [36]:
w = Window.partitionBy("arquivo", "id_eleitor").orderBy("data")
df_tempo_segundos = (
    df_teclas.withColumn("lag_tempo", F.lag(df_teclas["data"], 1).over(w))
    .withColumn(
        "tempo(segundos)",
        (
            F.unix_timestamp(
                df_teclas["data"],
            )
            - F.unix_timestamp(F.col("lag_tempo"))
        ),
    )
)

In [38]:
df_tempo_segundos.count()

4186

In [42]:
df_tempo_segundos.show()

+-------------------+--------------------+--------------------+---------+--------------+---------------+-----------+----------+-------------------+---------------+
|               data|           descricao|             arquivo|Municipio|Zona_Eleitoral|Secao_Eleitoral|erros_tecla|id_eleitor|          lag_tempo|tempo(segundos)|
+-------------------+--------------------+--------------------+---------+--------------+---------------+-----------+----------+-------------------+---------------+
|2022-10-02 08:01:46|Eleitor foi habil...|file:///app/data/...|    17256|            44|            147|          0|         1|               NULL|           NULL|
|2022-10-02 08:02:26|Voto confirmado p...|file:///app/data/...|    17256|            44|            147|          0|         1|2022-10-02 08:01:46|             40|
|2022-10-02 08:02:44|Voto confirmado p...|file:///app/data/...|    17256|            44|            147|          0|         1|2022-10-02 08:02:26|             18|
|2022-10-02 08:0

In [41]:
df_tempo_segundos.write.mode('overwrite').parquet(DATA_PATH+"dados_tratados.parquet")

In [ ]:
sc.stop()